In [ ]:
import os
import platform

if not os.path.exists("./datasets.zip"):
    print("upload datasets")
elif platform.system() == "Windows":
    os.system("git clone https://github.com/n1teshy/poet & move poet/tokenizer . & move poet/core . & rd /s /q poet")
    os.system("powershell Expand-Archive -Path ./datasets -DestinationPath .")
else:
    os.system("git clone https://github.com/n1teshy/poet && mv poet/tokenizer . && mv poet/core . && rm -rf poet")
    os.system("unzip ./datasets -d .")

In [1]:
import torch

import torch.nn.functional as F
from torch.optim import AdamW
from core.tokenizers.regex import get_tokenizer
from core.utils import get_param_count
from core.config import device
from core.models import Generator

In [2]:
BLOCK_SIZE = 32
BATCH_SIZE = 64
EPOCHS = 10
EMBEDDING_SIZE = 256
LAYERS = 1
HEADS = 32
TRAIN_FILE = "datasets/train.txt"
TEST_FILE = "datasets/test.txt"

In [5]:
tokenizer = get_tokenizer("poems.txt", 1024, "tokenizer/en", True)
train_data = tokenizer.encode(open(TRAIN_FILE, encoding="utf-8").read())
test_data = tokenizer.encode(open(TEST_FILE, encoding="utf-8").read())

In [17]:
def get_batch(split):
    data = train_data if split == "train" else test_data
    idxs = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE, ))
    X = [data[idx: idx + BLOCK_SIZE] for idx in idxs]
    Y = [data[idx + 1: idx + 1 + BLOCK_SIZE] for idx in idxs]
    return torch.tensor(X, device=device), torch.tensor(Y, device=device)

In [ ]:
model = Generator(tokenizer.size, EMBEDDING_SIZE, BLOCK_SIZE, LAYERS, HEADS)
print("%.4f mn parameters" % (get_param_count(model) / 1e6, ))

In [28]:
@torch.no_grad()
def get_test_loss():
    model.eval()
    inp, tgt = get_batch("test")
    logits = model(inp)
    B, T, C = logits.shape
    logits, tgt = logits.reshape(B*T, C), tgt.reshape(B*T)
    loss = F.cross_entropy(logits, tgt)
    model.train()
    return loss


In [18]:
def batch_generator(split):
    while True:
        yield get_batch(split)

In [36]:
optimizer = AdamW(model.parameters(), lr=0.001)

In [37]:
mean_train_loss, mean_test_loss = 0, 0
batch_to_epoch = len(train_data) / BATCH_SIZE

In [ ]:
for batch_no, (inp, tgt) in enumerate(batch_generator("train"), start=1):
    optimizer.zero_grad()
    logits = model(inp)
    B, T, C = logits.shape
    logits, tgt = logits.reshape(B * T, C), tgt.reshape(B * T)
    train_loss = F.cross_entropy(logits, tgt)
    train_loss.backward()
    optimizer.step()
    test_loss = get_test_loss()
    train_loss, test_loss = train_loss.item(), test_loss.item()
    mean_train_loss = (mean_train_loss or train_loss) * 0.99 + train_loss * 0.01
    mean_test_loss = (mean_test_loss or test_loss) * 0.99 + test_loss * 0.01
    print(
        "%d:%d -> (%.4f | %.4f), (%.4f | %.4f)"
        % (
            batch_no // batch_to_epoch + 1,
            batch_no % batch_to_epoch,
            train_loss,
            mean_train_loss,
            test_loss,
            mean_test_loss,
        )
    )
    
